In [1]:
import pandas as pd
pred = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/data/df_prepro.csv", index_col=0)

In [45]:
pred.dtypes

Home                           int64
Away                           int64
FTR                            int64
Attendance                   float64
B365H                        float64
B365D                        float64
B365A                        float64
AvgHomePoints                float64
AvgAwayPoints                float64
AvgLosingHomePoints          float64
AvgLosingAwayPoints          float64
AvgGoalDiffHome              float64
AvgGoalDiffAway              float64
HomeWinsRatio                float64
HomeDrawsRatio               float64
AwayWinsRatio                float64
AwayDrawsRatio               float64
AvgHomeCornersLast5          float64
AvgAwayCornersLast5          float64
AvgHomeShotsLast5            float64
AvgHomeShotsOnTargetLast5    float64
AvgAwayShotsLast5            float64
AvgAwayShotsOnTargetLast5    float64
elo                          float64
elo_away                     float64
FormHomeTeam                 float64
FormAwayTeam                 float64
P

In [2]:
pred['Home'] = pred['Home'].astype('category')
pred['Away'] = pred['Away'].astype('category')
#pred['FTR'] = pred['FTR'].astype('category')

In [3]:
pred["target"] = (pred["FTR"])

In [4]:
pred['Date'] = pd.to_datetime(pred['Date'], format='%Y-%m-%d')

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [7]:
train = pred[pred["Date"] < '2024-01-01']
test = pred[pred["Date"] > '2024-01-01']

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [9]:
predictors = [
    'Home', 'Away', 'Attendance', 
    'B365H', 'B365D', 'B365A', 'AvgHomePoints', 'AvgAwayPoints', 
    'AvgLosingHomePoints', 'AvgLosingAwayPoints', 'AvgGoalDiffHome', 
    'AvgGoalDiffAway', 'HomeWinsRatio', 'HomeDrawsRatio', 'AwayWinsRatio', 'AwayDrawsRatio', 
    'AvgHomeCornersLast5', 'AvgAwayCornersLast5', 'AvgHomeShotsLast5', 'AvgHomeShotsOnTargetLast5', 
    'AvgAwayShotsLast5', 'AvgAwayShotsOnTargetLast5', 'elo', 'elo_away','FormHomeTeam', 
    'FormAwayTeam', 'ProbabilityHomeWin', 'ProbabilityAwayWin'
]

In [54]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), predictors)
    ])

train = preprocessor.fit_transform(train)
test = preprocessor.transform(test)

In [10]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [11]:
preds_train = rf.predict(train[predictors])

In [12]:
preds_test = rf.predict(test[predictors])

In [13]:
from sklearn.metrics import accuracy_score

In [16]:
acc_train = accuracy_score(train["target"], preds_train)

In [17]:
acc_train

0.9535294117647058

In [19]:
combined_train = pd.DataFrame(dict(actual=train["target"], predicted=preds_train))

In [21]:
pd.crosstab(index=combined_train["actual"], columns=combined_train["predicted"])

predicted,-1,0,1
actual,,,
-1,431,3,37
0,6,446,19
1,12,2,744


In [22]:
acc = accuracy_score(test["target"], preds_test)

In [23]:
acc

0.53125

In [25]:
combined_test = pd.DataFrame(dict(actual=test["target"], predicted=preds_test))

In [26]:
pd.crosstab(index=combined_test["actual"], columns=combined_test["predicted"])

predicted,-1,0,1
actual,,,
-1,23,7,19
0,7,6,25
1,9,8,56
